# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [16]:
# imports
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama

In [10]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [11]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

openai = OpenAI()

API key looks good so far


In [12]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [13]:
# prompts

system_prompt = "You are a helpful technical tutor who answers questions about python code, software engineering, data science and LLMs"
user_prompt = "Please give a detailed explanation to the following question: " + question

In [14]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [15]:
# Get gpt-4o-mini to answer, with streaming

stream = openai.chat.completions.create(model=MODEL_GPT, messages=messages,stream=True)
    
response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [17]:
# Get Llama 3.2 to answer

response = ollama.chat(model=MODEL_LLAMA, messages=messages)
reply = response['message']['content']
display(Markdown(reply))

**Code Explanation**

The given code snippet is written in Python and utilizes generator expression to extract author names from a list of dictionaries.

```python
yield from {book.get("author") for book in books if book.get("author")}
```

Let's break it down:

- `for book in books`: This part iterates over each dictionary (`book`) in the `books` collection. The `books` variable is expected to be a list or iterable containing dictionaries.

- `if book.get("author")`: This condition filters out any dictionaries that do not have an `"author"` key. If a dictionary does not have this key, its value will be `None`, which is considered `False` in a boolean context. Therefore, such dictionaries are skipped during the iteration.

- `{book.get("author") for book in books if book.get("author")}`: This part generates an iterator (generator) that yields author names from the filtered dictionaries. The `get()` method of each dictionary attempts to retrieve the value associated with the `"author"` key, and returns `None` if it does not exist.

- `yield from ...`: This is a Python 3.3+ feature that allows us to delegate iteration to another iterator or generator expression. In this case, we're yielding each author name directly using the dictionary's `.get("author")`, but then delegating the actual iteration to the inner generator expression.

**Why it works**

The key benefits of using `yield from` are:

- **Lazy evaluation**: Instead of creating an entire list or set of authors and then iterating over it, we generate each author name on demand. This can be beneficial if you're working with a large dataset.
- **Efficient memory usage**: By not storing all the author names in memory at once, we minimize memory allocation and deallocation overhead.

However, the `yield from` expression itself doesn't change the logic of how data is extracted from the dictionaries. It simply makes it more concise and easier to read.

**Alternative implementation**

If you're working with Python 3.10 or later, you can use the following alternative syntax for achieving similar results:

```python
authors = [book.get("author") for book in books if book.get("author")]
```

This version of the code is more straightforward and doesn't require `yield from`. However, it does create a list of authors as an intermediate step, which may not be desirable for very large datasets. The original `yield from` implementation addresses this by generating each author name on demand.